# 01 - Exploratory Data Analysis
## Fresh Flow Markets - Demand Forecasting

This notebook explores the order data from Fresh Flow Markets (Copenhagen restaurant chain) to understand:
- Order volume trends (Feb 2021 - Feb 2024)
- Top items per store
- Daily/weekly/monthly demand patterns
- Day-of-week and seasonality
- Promotion impact
- Data quality issues

In [ ]:

import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from src.data.loader import load_key_tables, load_config
from src.data.cleaner import clean_all

pd.set_option('display.max_columns', 50)
config = load_config()
tables = load_key_tables(config)
tables = clean_all(tables)

orders = tables['fct_orders']
items = tables['fct_order_items']
dim_items = tables['dim_items']
places = tables['dim_places']
campaigns = tables['fct_campaigns']

print(f'Orders: {len(orders):,} rows')
print(f'Order items: {len(items):,} rows')
print(f'Items catalog: {len(dim_items):,} rows')
print(f'Places: {len(places):,} rows')
print(f'Campaigns: {len(campaigns):,} rows')

## 1. Order Volume Trends

In [ ]:
# Daily order volume over time
orders['date'] = orders['created_dt'].dt.date
daily_orders = orders.groupby('date').size().reset_index(name='order_count')
daily_orders['date'] = pd.to_datetime(daily_orders['date'])

fig = px.line(daily_orders, x='date', y='order_count', title='Daily Order Volume (All Stores)')
fig.update_layout(template='plotly_white')
fig.show()

# Monthly aggregation
daily_orders['month'] = daily_orders['date'].dt.to_period('M').dt.start_time
monthly = daily_orders.groupby('month')['order_count'].sum().reset_index()
fig = px.bar(monthly, x='month', y='order_count', title='Monthly Order Volume')
fig.update_layout(template='plotly_white')
fig.show()

## 2. Store-Level Analysis

In [ ]:
# Orders per store
store_orders = orders.merge(places[['id', 'title']], left_on='place_id', right_on='id', suffixes=('', '_place'))
store_counts = store_orders.groupby('title').size().reset_index(name='order_count').sort_values('order_count', ascending=True)

fig = px.bar(store_counts, x='order_count', y='title', orientation='h', title='Total Orders by Store')
fig.update_layout(template='plotly_white', yaxis_title='')
fig.show()

# Revenue per store
store_revenue = store_orders.groupby('title')['total_amount'].sum().reset_index().sort_values('total_amount', ascending=True)
fig = px.bar(store_revenue, x='total_amount', y='title', orientation='h', title='Total Revenue by Store (DKK)')
fig.update_layout(template='plotly_white', yaxis_title='')
fig.show()

## 3. Top Items Analysis

In [ ]:
# Top items by total quantity
item_demand = items.groupby(['item_id', 'title'])['quantity'].sum().reset_index()
item_demand = item_demand.sort_values('quantity', ascending=False).head(20)

fig = px.bar(item_demand, x='quantity', y='title', orientation='h', title='Top 20 Items by Total Quantity')
fig.update_layout(template='plotly_white', yaxis_title='', yaxis={'categoryorder': 'total ascending'})
fig.show()

## 4. Day-of-Week and Seasonal Patterns

In [ ]:
# Day-of-week pattern
orders['dow'] = orders['created_dt'].dt.dayofweek
dow_names = {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'}
dow_counts = orders.groupby('dow').size().reset_index(name='count')
dow_counts['day'] = dow_counts['dow'].map(dow_names)

fig = px.bar(dow_counts, x='day', y='count', title='Orders by Day of Week')
fig.update_layout(template='plotly_white')
fig.show()

# Monthly seasonality
orders['month_num'] = orders['created_dt'].dt.month
month_counts = orders.groupby('month_num').size().reset_index(name='count')
fig = px.bar(month_counts, x='month_num', y='count', title='Orders by Month')
fig.update_layout(template='plotly_white', xaxis_title='Month')
fig.show()

## 5. Data Quality Check

In [ ]:
print('=== Data Quality Summary ===')
print(f'\nOrders date range: {orders["created_dt"].min()} to {orders["created_dt"].max()}')
print(f'Order statuses: {orders["status"].value_counts().to_dict()}')
print(f'\nNull values in key columns:')
for col in ['place_id', 'total_amount', 'status', 'created_dt']:
    if col in orders.columns:
        print(f'  orders.{col}: {orders[col].isna().sum()}')
for col in ['item_id', 'order_id', 'quantity', 'cost']:
    if col in items.columns:
        print(f'  order_items.{col}: {items[col].isna().sum()}')
print(f'\nDuplicate order IDs: {orders["id"].duplicated().sum()}')
print(f'Duplicate order item IDs: {items["id"].duplicated().sum()}')